In [57]:
import collections
import itertools
from scipy.sparse import random
import numpy as np

# The Algorithm

In [83]:
# input:
# A is a numpy matrix mxn
# P_c is a permutation matrix as an int ndarray of shape (N,)
# k is an integer
# output:
# Q_k is a numpy matrix
# R_k is a numpy matrix
def lu_crtp_4(A, P_c, k):
    AP_c = A[:, P_c]
    AP_c_selected_columns = AP_c[:, :k]
    Q_k, R_k = np.linalg.qr(AP_c_selected_columns)
    return Q_k, R_k

# input:
# a is a numpy matrix mxn
# k is an even integer
def lu_crtp(a, k, debug=False):
    # According to https://epubs-siam-org.uaccess.univie.ac.at/doi/epdf/10.1137/13092157X QR_TP is the same
    # as RRQR. For a faster implementation, we therefore decided on using scipy's LAPACK interface.
    # Select k columns by using QR with tournament pivoting on A
    _, _, p_c = qr(a, pivoting=True)
    #p_c = p_c[:k]


    # Compute the thin QR factorization of the selected columns
    q_k, r_k = lu_crtp_4(a, p_c, k)
    if(debug):
        print("q_k:")
        print(q_k.shape)
        print("r_k:")
        print(r_k.shape)


    # Select k rows by using QR with tournament pivoting on Q^T_k
    _, _, p_r = qr(q_k.T, pivoting=True)
    #p_r = p_r[:k]

    # Let A_ = P ...
    a_dash = a[p_r,:]
    a_dash = a_dash[:,p_c]
    rows, cols = a_dash.shape
    if(debug):
        print("a_dash:")
        print(a_dash.shape)
    
    # Separate into block matrices
    a_dash_11 = a_dash[:k, :k]
    a_dash_21 = a_dash[k:, :k]
    a_dash_12 = a_dash[:k, k:]
    if(debug):
        print("a_dash_11:")
        print(a_dash_11.shape)
        print("a_dash_21:")
        print(a_dash_21.shape)
        print("a_dash_12:")
        print(a_dash_12.shape)


    # Compute L_21
    inv_a_dash_11 = np.linalg.inv(a_dash_11)
    l_21 = np.dot(a_dash_21, inv_a_dash_11)
    if(debug):
        print("l_21:")
        print(l_21.shape)

    
    # Stack the block matrices
    i = np.identity(k)
    l_k = np.vstack((i, l_21))
    if(debug):
        print("l_k:")
        print(l_k.shape)
    u_k = np.hstack((a_dash_11, a_dash_12))
    if(debug):
        print("u_k:")
        print(u_k.shape)
    return p_r, p_c, l_k, u_k, r_k

# returns true if A and the approximation is equal
def compareApprox(A, p_r, p_c, l_k, u_k):
    approx = np.dot(l_k, u_k)
    print("approx:")
    print(approx)
    PA = p_A = A[p_r, :]
    PA = p_A[:, p_c]
    print("PA:")
    print(PA)
    return np.allclose(approx, PA)

# Recommender System

In [59]:
def Recommender_System(orders, min_support, item_threshold=0.5):
    mapped_records = []
    # In the following tasks use the mapped records to compute the frequent itemsets.
    for record in orders:
        mapped_record = []
        for index, item in enumerate(record):
            if(item > item_threshold):
                mapped_record.append(index)
        mapped_record.sort()
        mapped_records.append(mapped_record)

        # Apriori Algorithm
        l1_items = collections.Counter()
        for record in mapped_records:
            l1_items.update(record)
        frequent_l1_items = {}
        for item in l1_items:
            support = l1_items[item]
            if support >= min_support:
                frequent_l1_items[(item,)] = support
        frequent_itemsets = {}
        for item in frequent_l1_items:
            frequent_itemsets[item] = frequent_l1_items[item]
        frequent_n_itemsets = frequent_l1_items
        
    def apriori_gen(itemsets):
        C_k = set()
        for p in itemsets:
            for q in itemsets:
                if p[-1] < q[-1]:
                    C_k.add( p + (q[-1],) )
        def all_subsets_in_itemsets(x):
            for subset in itertools.combinations(x, len(x) - 1):
                if subset not in itemsets:
                    return False
            return True
        return list(filter(all_subsets_in_itemsets, C_k))

    def calculate_support(itemset):
        if len(itemset) == 1:
            try:
                return frequent_l1_items[itemset]
            except KeyError:
                return 0
        support = 0
        for record in mapped_records:
            itemset_in_record = True
            for item in itemset:
                if item not in record:
                    itemset_in_record = False
                    break
            if itemset_in_record:
                support += 1
        return support

    while len(frequent_n_itemsets) != 0:
        candidates = apriori_gen(frequent_n_itemsets)
        supports = map(calculate_support, candidates)
        frequent_candidates = {}
        for candidate, support in zip(candidates, supports):
            if support >= min_support:
                frequent_candidates[candidate] = support
        for item in frequent_candidates:
            frequent_itemsets[item] = frequent_candidates[item]
        frequent_n_itemsets = [itemset for itemset in frequent_candidates]

    return frequent_itemsets

In [90]:
def print_support(supports):
    for itemset in supports:
        support = supports[itemset]
        print(f"{itemset} : {support}")
        
def compare_supports(supports1, supports2):
    dist = 0
    for itemset in supports1:
        support1 = supports1[itemset]
        support2 = supports2.get(itemset, 0)
        dist = dist + np.square(support1 - support2)
    for itemset in supports2:
        support2 = supports2[itemset]
        if(itemset not in supports1):
            dist = dist + np.square(support2)            
    return np.sqrt(dist)
    

# Generate Random Matrix

In [97]:
shape = (25, 20)
if(shape[0] < shape[1]):
    print("we need more users than items to use the approximation for the recommender system!")
density = 0.2
rng = np.random.default_rng(1234);

random_matrix = random(*shape, density=density, format='csr', random_state=rng)
random_matrix.data[:] = 1
original = random_matrix.toarray()

# Execution

In [98]:
k = 1
p_r, p_c, l_k, u_k, r_k = lu_crtp(original, k, debug=False)
approx = np.dot(l_k, u_k)

pivoted_original = original[:, p_c]
rounded_approx = np.where(approx > 0.5, 1, 0)

In [99]:
print(original)
print(pivoted_original)

[[0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.

In [86]:
min_support = 10;
original_support = Recommender_System(pivoted_original, min_support)
approx_support = Recommender_System(rounded_approx, min_support)

In [88]:
print(f"{pivoted_original.shape=}")
print(f"{rounded_approx.shape=}")
print("")
print(compare_supports(original_support, approx_support))

pivoted_original.shape=(50, 25)
rounded_approx.shape=(50, 25)

support1=14 support2=14 dist=0
support1=13 support2=13 dist=0
support1=11 support2=11 dist=0
support1=12 support2=12 dist=0
support1=10 support2=10 dist=0
support1=11 support2=11 dist=0
support1=10 support2=10 dist=0
support1=11 support2=11 dist=0
support1=10 support2=10 dist=0
support1=12 support2=12 dist=0
support1=13 support2=13 dist=0
support1=11 support2=11 dist=0
support1=10 support2=10 dist=0
support1=10 support2=10 dist=0
support1=12 support2=14 dist=4
support1=10 support2=10 dist=4
support1=10 dist=229
support1=10 dist=373
support1=10 dist=517
22.737634001804146


In [93]:
print("original_support:")
print_support(original_support)
print("approx_support:")
print_support(approx_support)

original_support:
(0,) : 14
(2,) : 13
(7,) : 11
(8,) : 12
(12,) : 10
(5,) : 11
(17,) : 10
(10,) : 11
(11,) : 10
(1,) : 12
(3,) : 13
(4,) : 11
(6,) : 10
(9,) : 10
(20,) : 12
(16,) : 10
approx_support:
(1,) : 12
(2,) : 13
(5,) : 11
(8,) : 12
(10,) : 11
(23,) : 15
(6,) : 10
(17,) : 10
(20,) : 14
(0,) : 14
(9,) : 10
(16,) : 10
(22,) : 12
(3,) : 13
(7,) : 11
(21,) : 12
(4,) : 11
(11,) : 10
(12,) : 10
